In [ ]:
import numpy as np 
import pandas as pd 
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        
database='/kaggle/input/soccer/database.sqlite'

## First we will create the connection to the db, and see what tables we have 

In [ ]:
conn=sqlite3.connect(database)

tables=pd.read_sql(""" SELECT * FROM sqlite_master WHERE type='table';""",conn)
tables

### List of Countries

In [ ]:
countries=pd.read_sql("""SELECT * FROM country;""",conn)
countries

### List of leagues and their country

JOIN is used when you want to connect two tables. it works when you have a common key in eoach of them. 

When joining between different tables, you must:

* Decide what type of join to use.The most common are:
   * **(INNER)** JOIN: keep only records that matche the condition in both tables, and records in both tables that do not match wouldn't appear in the output.
   * **(LEFT)** JOIN: keep all the values from the first  (left) table - in the conjuction with the matching rows from the right table. The columns from the the second(right) table. the columns from the right table, that don't have matching value in the left, would have NULL values.

* Make sure that at least one of the values have to be a key in its table. in our case, it's the **Country.id**. the leaguge.Country_id is not unique, as there can be more than one leauge in the same country.

In [ ]:
leagues=pd.read_sql("""SELECT * FROM League JOIN Country On Country.id=League.Country_id;""",conn)
leagues

### List of Teams           

ORDER BY defines the sorting of the output - ascending  or descinding

LIMIT, limits the number of rows in the output - after the sorting

In [ ]:
teams=pd.read_sql("""SELECT * From Team ORDER BY team_long_name LIMIT 10;""",conn)
teams

### List of matches                     

Now we will trybshow only the columns that interest us, so instead `*` we will use the excat names.

Some of the cells have the same names (Country.name, League.name). We will rename.

In [ ]:
detailed_matches = pd.read_sql("""SELECT Match.id, 
                                        Country.name AS country_name, 
                                        League.name AS league_name, 
                                        season, 
                                        stage, 
                                        date,
                                        HT.team_long_name AS  home_team,
                                        AT.team_long_name AS away_team,
                                        home_team_goal, 
                                        away_team_goal                                        
                                FROM Match
                                JOIN Country on Country.id = Match.country_id
                                JOIN League on League.id = Match.league_id
                                LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
                                LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
                                ORDER by date;""", conn)
detailed_matches

## Top 10 Teams with most number of goals [home_team_goal +  away_team_goal]

In [ ]:
team_goals=pd.read_sql("""SELECT t.team_long_name, SUM(m.home_team_goal + m.away_team_goal) AS total_goals
FROM Team t
JOIN Match m ON t.id = m.id
GROUP BY t.team_long_name
ORDER BY total_goals DESC
LIMIT 10;""",conn)
team_goals

## Which team has the highest average deffence pressure ? 

In [ ]:
avg_defence_pressure=pd.read_sql("""SELECT t.team_long_name, avg(ta.defencePressure) AS avg_defence_pressure 
From Team t JOIN Team_Attributes ta ON t.team_api_id = ta.team_api_id
GROUP BY t.team_long_name
ORDER BY avg_defence_pressure DESC
LIMIT 10;""",conn)
avg_defence_pressure

## Which player has highest average overall rating with total no. of seasons ?

In [ ]:
avg_rating_by_players=pd.read_sql("""SELECT p.player_name, 
AVG(pa.overall_rating) AS avg_overall_rating,
COUNT(distinct m.season) AS total_seasons
FROM Player p JOIN Match m ON p.id= m.id 
JOIN Player_Attributes pa ON pa.player_api_id = p.player_api_id
GROUP BY p.player_name
ORDER BY avg_overall_rating DESC
LIMIT 10;""",conn)
avg_rating_by_players

In [ ]:
plt.figure(figsize=(12,6))
sns.barplot(x=avg_rating_by_players['player_name'],y=avg_rating_by_players['avg_overall_rating'])
plt.title('Player Average OVerall Rating')
plt.xticks(rotation=45)
plt.show()

## Let's do some basic analysis  

In [ ]:
leagues_by_season = pd.read_sql("""SELECT Country.name AS country_name, 
                                        League.name AS league_name, 
                                        season,
                                        count(distinct stage) AS number_of_stages,
                                        count(distinct HT.team_long_name) AS number_of_teams,
                                        avg(home_team_goal) AS avg_home_team_scors, 
                                        avg(away_team_goal) AS avg_away_team_goals, 
                                        avg(home_team_goal-away_team_goal) AS avg_goal_dif, 
                                        avg(home_team_goal+away_team_goal) AS avg_goals, 
                                        sum(home_team_goal+away_team_goal) AS total_goals                                       
                                FROM Match
                                JOIN Country on Country.id = Match.country_id
                                JOIN League on League.id = Match.league_id
                                LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
                                LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
                                WHERE country_name in ('Spain', 'Germany', 'France', 'Italy', 'England')
                                GROUP BY Country.name, League.name, season
                                HAVING count(distinct stage) > 10
                                ORDER BY Country.name, League.name, season DESC
                                ;""", conn)
leagues_by_season

In [ ]:
df=pd.DataFrame(index=np.sort(leagues_by_season['season'].unique()),columns=leagues_by_season['country_name'].unique())

df.loc[:,'Germany']=list(leagues_by_season.loc[leagues_by_season['country_name']=='Germany','avg_goals'])
df.loc[:,'Spain']=list(leagues_by_season.loc[leagues_by_season['country_name']=='Spain','avg_goals'])
df.loc[:,'France']=list(leagues_by_season.loc[leagues_by_season['country_name']=='France','avg_goals'])
df.loc[:,'Italy']=list(leagues_by_season.loc[leagues_by_season['country_name']=='Italy','avg_goals'])
df.loc[:,'England']=list(leagues_by_season.loc[leagues_by_season['country_name']=='England','avg_goals'])

df.plot(figsize=(12,6),title='Average Goals per Game Over Time')